In [1]:
import glob
import numpy as np
import copy
import math

filepaths_to_be_wrapped = []
for i in range(1, 2):
    for file_name in set(glob.glob(str(i) + 'run/*.pdb')) - set(glob.glob(str(i) + 'run/*wrapped*')):
        filepaths_to_be_wrapped.append(file_name)
print(filepaths_to_be_wrapped)


['1run\\118000_checkpoint.pdb', '1run\\snapshot_2.pdb']


In [2]:
for file_name in filepaths_to_be_wrapped:
    temp_newfilename = file_name.split('.')
    newfilename = temp_newfilename[0] + '_wrapped.' + temp_newfilename[1]
    with open(file_name, 'r') as f:
        water_index_list, acrolein_index_list = [], []
        lines = f.readlines()
        for i in range(len(lines)):
            if 'CRYST1' in lines[i]:
                need = lines[i].split()
                x_bound, y_bound, z_boudn = float(need[1]), float(need[2]), float(need[3])
            if 'ATOM' in lines[i]:
                if ' O ' in lines[i]:
                    if ' H ' in lines[i+1]:
                        water_index_list.append(i)
                    if ' C ' in lines[i+1]:
                        acrolein_index_list.append(i)
            f.close()

In [3]:
def subtract_atom(molecule, index, bounds):
    for i in range(len(molecule)):
        molecule[i][index] = round(float(molecule[i][index]) - bounds, 3)
    return molecule

def add_atom(molecule, index, bounds):
    for i in range(len(molecule)):
        molecule[i][index] = round(float(molecule[i][index]) + bounds, 3)
    return molecule

def atom_together(atom_1, atom_2):
    dist0 = atom_1[0] - atom_2[0]
    dist1 = atom_1[1] - atom_2[1]
    dist2 = atom_1[2] - atom_2[2]
    if abs(dist0) > 5 or abs(dist1) > 5 or abs(dist2) > 5:
        return False
    else:
        return True

def dist(atom_dim_1, atom_dim_2):
    return round(atom_dim_1 - atom_dim_2, 3)

def align(atom_a, atom_b, atom_c):

    xab = math.isclose(atom_a[0], atom_b[0], abs_tol = 5)
    yab = math.isclose(atom_a[1], atom_b[1], abs_tol = 5)
    zab = math.isclose(atom_a[2], atom_b[2], abs_tol = 5)
    xac = math.isclose(atom_a[0], atom_c[0], abs_tol = 5)
    yac = math.isclose(atom_a[1], atom_c[1], abs_tol = 5)
    zac = math.isclose(atom_a[2], atom_c[2], abs_tol = 5)
    xbc = math.isclose(atom_b[0], atom_c[0], abs_tol = 5)
    ybc = math.isclose(atom_b[1], atom_c[1], abs_tol = 5)
    zbc = math.isclose(atom_b[2], atom_c[2], abs_tol = 5)
    #print(xab, xac, xbc)
    #print(yab, yac, ybc)
    #print(zab, zac, zbc)
    if xab == False or xac == False or xbc == False :
        if xab and xac == False:
            print('xc is the problem')
            print(atom_a, atom_b, atom_c)
            if atom_c[0] < atom_b[0] and atom_c[0] < atom_a[0]:
                atom_c[0] = round(atom_c[0] + x_bound, 3)
            elif atom_c[0] > atom_b[0] and atom_c[0] > atom_a[0]:
                atom_c[0] = round(atom_c[0] - x_bound, 3)
            #print(atom_a[0])
            #print(atom_b[0])
            #print(atom_c[0])
        elif xac and xbc == False:
            print('xb is the problem')
            print(atom_a, atom_b, atom_c)
            if atom_b[0] < atom_a[0] and atom_b[0] < atom_c[0]:
                atom_b[0] = round(atom_b[0] + x_bound, 3)
            elif atom_b[0] > atom_a[0] and atom_b[0] > atom_c[0]:
                atom_b[0] = round(atom_b[0] - x_bound, 3)
            #print(atom_a[0])
            #print(atom_b[0])
            #print(atom_c[0])

        elif xab and xbc == False:
            print('xa is the problem')
            print(atom_a, atom_b, atom_c)
            if atom_a[0] < atom_b[0] and atom_a[0] < atom_c[0]:
                atom_a[0] = round(atom_a[0] + x_bound, 3)
            elif atom_a[0] > atom_b[0] and atom_a[0] > atom_c[0]:
                atom_a[0] = round(atom_a[0] - x_bound, 3)
            #print(atom_a[0])
            #print(atom_b[0])
            #print(atom_c[0])

    elif yab == False or yac == False or ybc == False:
        if yab and yac == False:
            print('yc is the problem')
            print(atom_a, atom_b, atom_c)
            if atom_c[1] < atom_b[1] and atom_c[1] < atom_a[1]:
                atom_c[1] = round(atom_c[1] + x_bound, 3)
            elif atom_c[1] > atom_b[1] and atom_c[1] > atom_a[1]:
                atom_c[1] = round(atom_c[1] - x_bound, 3)
            #print(atom_a[1])
            #print(atom_b[1])
            #print(atom_c[1])
        elif yac and ybc == False:
            print('yb is the problem')
            print(atom_a, atom_b, atom_c)
            if atom_b[1] < atom_a[1] and atom_b[1] < atom_c[1]:
                atom_b[1] = round(atom_b[1] + x_bound, 3)
            elif atom_b[1] > atom_a[1] and atom_b[1] > atom_c[1]:
                atom_b[1] = round(atom_b[1] - x_bound, 3)
            #print(atom_a[1])
            #print(atom_b[1])
            #print(atom_c[1])
        elif yab and ybc == False:
            print('ya is the problem')
            print(atom_a, atom_b, atom_c)
            if atom_a[1] < atom_b[1] and atom_a[1] < atom_c[1]:
                atom_a[1] = round(atom_a[1] + x_bound, 3)
            elif atom_a[1] > atom_b[1] and atom_a[1] > atom_c[1]:
                atom_a[1] = round(atom_a[1] - x_bound, 3)
            #print(atom_a[1])
            #print(atom_b[1])
            #print(atom_c[1])
    elif zab == False or zac == False or zbc == False:
        if zab and zac == False:
            print('zc is the problem')
            print(atom_a, atom_b, atom_c)
            if atom_c[2] < atom_b[2] and atom_c[2] < atom_a[2]:
                atom_c[2] = round(atom_c[2] + x_bound, 3)
            elif atom_c[2] > atom_b[2] and atom_c[2] > atom_a[2]:
                atom_c[2] = round(atom_c[2] - x_bound, 3)
            #print(atom_a[2])
            #print(atom_b[2])
            #print(atom_c[2])
        elif zac and zbc == False:
            print('zb is the problem')
            print(atom_a, atom_b, atom_c)
            if atom_b[2] < atom_a[2] and atom_b[2] < atom_c[2]:
                atom_b[2] = round(atom_b[2] + x_bound, 3)
            elif atom_b[2] > atom_a[2] and atom_b[2] > atom_c[2]:
                atom_b[2] = round(atom_b[2] - x_bound, 3)
            #print(atom_a[2])
            #print(atom_b[2])
            #print(atom_c[2])
        elif zbc and zab == False:
            print('za is the problem')
            print(atom_a, atom_b, atom_c)
            if atom_a[2] < atom_b[2] and atom_a[2] < atom_c[2]:
                atom_a[2] = round(atom_a[2] + x_bound, 3)
            elif atom_a[2] > atom_b[2] and atom_a[2] > atom_c[2]:
                atom_a[2] = round(atom_a[2] - x_bound, 3)
            #print(atom_a[2])
            #print(atom_b[2])
            #print(atom_c[2])
    return atom_a, atom_b, atom_c
    

def check_together(atom_a, atom_b, atom_c):
    atom_a_atom_b_together = atom_together(atom_1 = atom_a, atom_2 = atom_b)
    atom_a_atom_c_together = atom_together(atom_1 = atom_a, atom_2 = atom_c)
    atom_b_atom_c_together = atom_together(atom_1 = atom_b, atom_2 = atom_c)
    if atom_a_atom_b_together == False or atom_a_atom_c_together == False or atom_b_atom_c_together == False:
        return False
    else:
        return True

In [6]:
molecules = []
falses = []
trues = []
translated_molecules = []
for i in range(len(water_index_list)):
    molecule = []
    out_of_bounds_list = []
    for x in range(3):
        atom = lines[water_index_list[i] + x].split()
        atom = [float(atom[u]) for u in range(5, 8)]
        molecule.append(atom)
    molecules.append(molecule)
old_molecules = copy.deepcopy(molecules)
together_molecules = copy.deepcopy(molecules)
for x in range(len(molecules)):
    together = check_together(atom_a = molecules[x][0], atom_b = molecules[x][1], atom_c = molecules[x][2])
    if together == False:
        print(x)
        atom_a, atom_b, atom_c = align(atom_a = molecules[x][0], atom_b = molecules[x][1], atom_c = molecules[x][2])
        together_molecules[x][0], together_molecules[x][1], together_molecules[x][2] = atom_a, atom_b, atom_c
        falses.append('False')
    else:
        together_molecules[x] = molecules[x]
        trues.append('True')
    for y in range(len(molecules[x][0])):
        coordinate = together_molecules[x][0][y]
        if coordinate < 0:
            #print('old_molecule')
            #print(molecules[x])
            new_molecule = add_atom(molecule = together_molecules[x], index = y, bounds = x_bound)
            #print('new_molecule')
            #print(new_molecule)
        elif coordinate > x_bound:
            #print('old_molecule')
            new_molecule = subtract_atom(molecule = together_molecules[x], index = y, bounds = x_bound)
            #print('new_molecule')
            #print(new_molecule)
        else:
            new_molecule = together_molecules[x]
             
    translated_molecules.append(new_molecule)
#print(len(old_molecules))
#print(len(translated_molecules))
array_old_molecules= np.array(old_molecules)
array_translated_molecules = np.array(translated_molecules)
#print(array_translated_molecules - array_old_molecules)
#print('ayy')

#print(len(falses))
#print(len(trues))

15
yb is the problem
[1.883, 0.478, 20.892] [1.928, 24.944, 20.012] [2.177, 1.01, 21.489]
22
yc is the problem
[17.77, 25.008, -1.201] [17.193, 25.459, 23.839] [18.616, 0.606, 23.68]
27
zc is the problem
[8.881, 5.471, -1.535] [8.8, 4.645, -2.013] [8.181, 5.389, 23.466]
30
zb is the problem
[14.58, 12.061, 27.802] [14.273, 11.227, 3.911] [13.873, 12.068, 27.085]
39
xc is the problem
[3.998, 3.196, 17.373] [3.165, 3.534, 17.807] [29.083, 3.73, 17.753]
42
yb is the problem
[3.766, -2.246, 25.51] [4.497, 21.713, 25.007] [4.227, -1.514, 25.922]
44
zb is the problem
[9.213, 23.04, 25.906] [10.03, 23.152, 2.004] [8.637, 23.63, 26.357]
47
zb is the problem
[3.789, 7.056, 26.045] [4.232, 7.835, 1.275] [4.456, 6.478, 26.426]
51
61
xb is the problem
[22.751, 22.961, 14.903] [-1.799, 23.095, 14.058] [22.125, 22.775, 15.59]
87
xc is the problem
[-3.978, 19.348, 13.461] [-3.345, 18.862, 14.057] [19.583, 18.95, 13.615]
110
zc is the problem
[10.646, 10.842, 29.552] [11.479, 10.285, 29.708] [10.102, 

In [5]:
with open(newfilename, 'w') as f:
    new_lines = copy.deepcopy(lines)
    for i in range(len(lines)):
        new_lines[i] = lines[i].split()
    counter = 0

    for i in range(len(lines)):
        

        if i in water_index_list:
            new_lines[i][5] = str(translated_molecules[counter][0][0])
            new_lines[i][6] = str(translated_molecules[counter][0][1])
            new_lines[i][7] = str(translated_molecules[counter][0][2])
            new_lines[i+1][5] = str(translated_molecules[counter][1][0])
            new_lines[i+1][6] = str(translated_molecules[counter][1][1])
            new_lines[i+1][7] = str(translated_molecules[counter][1][2])
            new_lines[i+2][5] = str(translated_molecules[counter][2][0])
            new_lines[i+2][6] = str(translated_molecules[counter][2][1])
            new_lines[i+2][7] = str(translated_molecules[counter][2][2])
            counter += 1
        else:
            pass
    f.write(lines[0])
    f.write(lines[1])
    for i in range(2, len(new_lines)-1):
        f.write('{:<6}'.format(new_lines[i][0]) + '{:>5}'.format(new_lines[i][1]) + '{:>5}'.format(new_lines[i][2]) + '{:>4}'.format(new_lines[i][3]) + '{:>6}'.format(new_lines[i][4]) + '{:>12}'.format(new_lines[i][5]) + '{:>8}'.format(new_lines[i][6]) + '{:>8}'.format(new_lines[i][7]) + '{:>6}'.format(new_lines[i][8]) + '{:>6}'.format(new_lines[i][9]) + '{:>12}'.format(new_lines[i][10]) + '\n')

    f.write(lines[-1])